In [1]:
!ls -a

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

import utils.read_data as rd
import utils.io_model as io_m
import utils.preprocessing as pp

In [3]:
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path

In [4]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)

True

In [5]:
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH") + '/horse_racing'
DATA_PATH = GOOGLE_DRIVE_PATH + '/csv/'

In [6]:
print(GOOGLE_DRIVE_PATH)

C:\Users\endo_\Dropbox\workspace\HorseRacing/horse_racing


In [7]:
df = rd.read_horse_csv(DATA_PATH)

c:\users\endo_\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (13,61,98,109,146,157,182,194,205,230,242,253,258,278,279,280,290,301,306,326,327,328,338,349,354,373,374,375,376,377,378,379,380,381,382,383,386,390,393,394,395,397,399,402,404,405,406,407,408,409,410,411,412,413,421,422,423,424,425,426,427,428,429,430,431,432,434,438,441,442,443,445,447,450,452,453,454,455,456,457,458,459,460,461,469,472,473,474,475,476,477,478,479,480,486,489,490,491,495,500,501,502,503,504,505,506,507,508,509,517,520,521,522,523,524,525,526,527,528,534,537,538,539,543,548,549,550,551,552,553,554,555,556,557,565,568,569,570,571,572,573,574,575,576,582,585,586,587,591,596,597,598,599,600,601,602,603,604,605,617,618,620,621,622,623,624) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21327 entries, 0 to 21326
Columns: 769 entries, Unnamed: 0 to rentan3_15
dtypes: float64(384), int64(15), object(370)
memory usage: 125.1+ MB


In [9]:
df.head()

,Unnamed: 0,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,...,hukusyo_second_15,hukusyo_third_15,wakuren_15,umaren_15,wide_1_2_15,wide_1_3_15,wide_2_3_15,umatan_15,renhuku3_15,rentan3_15
0,0,2.020070e+11,6,7,8,2015100929,牝5,55.0,1117,1:52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.020060e+11,7,1,1,2015100929,牝5,52.0,1181,1:56.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2.020060e+11,9,5,5,2015100933,牝5,55.0,1170,1:56.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2.020060e+11,14,1,1,2015100933,牝5,52.0,1177,1:56.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2.020060e+11,3,1,1,2015100942,牡5,57.0,1127,1:11.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 前処理

In [10]:
# 学習に使用するカラム、過去データは3レース前までのデータを用いる
# これらのカラムに対して加工を行う為、新たなカラムが加わる。
# その為、最終的に用いる特徴量のカラムは別なセルで定義する(input_columns)
columns = ["race_course", "weather", "ground_status", 
           "where_racecourse", "race_class", "running_condition", 
           "frame_number", "horse_number",
           "sex_and_age", "burden_weight", "rider_id", 
           "tamer_id", "horse_weight", "odds", "popular",
           "rank", "total_horse_number_x", 
           "rank_1", "rank_2", "rank_3",
           "total_horse_number_x_1", "total_horse_number_x_2","total_horse_number_x_3",
           "goal_time_1", "goal_time_2", "goal_time_3",
           "last_time_1", "last_time_2", "last_time_3", 
           "half_way_rank_1", "half_way_rank_2", "half_way_rank_3", 
           "prize_1", "prize_2", "prize_3"]

df = df[columns]
df.head()

,race_course,weather,ground_status,where_racecourse,race_class,running_condition,frame_number,horse_number,sex_and_age,burden_weight,...,goal_time_3,last_time_1,last_time_2,last_time_3,half_way_rank_1,half_way_rank_2,half_way_rank_3,prize_1,prize_2,prize_3
0,ダ左1800m,天候 : 晴,ダート : 不良,1回中京8日目,4歳以上1勝クラス,牝[指](定量),7,8,牝5,55.0,...,NaN,40.3,NaN,NaN,10-9-5-6,NaN,NaN,NaN,NaN,NaN
1,ダ右1800m,天候 : 曇,ダート : 良,4回中山2日目,3歳以上1勝クラス,(混)[指](定量),1,1,牝5,52.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ダ右1800m,天候 : 曇,ダート : 稍重,4回中山1日目,3歳以上1勝クラス,[指](定量),5,5,牝5,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ダ右1800m,天候 : 曇,ダート : 重,4回中山6日目,3歳以上1勝クラス,(混)[指](定量),1,1,牝5,52.0,...,NaN,41.2,NaN,NaN,9-9-8-8,NaN,NaN,NaN,NaN,NaN
4,ダ右1200m,天候 : 曇,ダート : 稍重,4回中山7日目,3歳以上2勝クラス,(混)[指](定量),1,1,牡5,57.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df["kyakusitu_1"] = [nm.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x_1"].values, df["half_way_rank_1"])]

NameError: name 'nm' is not defined

In [ ]:
df["where_racecourse"] = df["where_racecourse"].map(pp.extract_place)

df["sex"] = df["sex_and_age"].map(lambda sex_and_age: sex_and_age[0])
df["age"] = df["sex_and_age"].map(lambda sex_and_age: sex_and_age[1:])

df["goal_time_1"] = df["goal_time_1"].map(pp.to_seconds)
df["goal_time_2"] = df["goal_time_1"].map(pp.to_seconds)
df["goal_time_3"] = df["goal_time_1"].map(pp.to_seconds)

df["horse_weight"] = df["horse_weight"].map(pp.extract_weight)

df["prize_1"] = df["prize_1"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df["prize_2"] = df["prize_2"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df["prize_3"] = df["prize_3"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)

df["kyakusitu_1"] = [nm.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x_1"].values, df["half_way_rank_1"])]
df["kyakusitu_2"] = [nm.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x_2"].values, df["half_way_rank_2"])]
df["kyakusitu_3"] = [nm.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x_3"].values, df["half_way_rank_3"])]

# 欠損値処理
df = df.replace('---', -1)
df = df.fillna(-1)

df["odds"] = df["odds"].astype(np.float64)

In [ ]:
# 欠損値の確認
df.isnull().sum().sum()

### ラベルの作成


In [ ]:
def make_label(rank_values, horse_number_values):
    labels = []
    high = 1 / 3
    mid = 2 / 3
    for rank, horse_number in zip(rank_values, horse_number_values):
        # 欠損値の場合
        if rank == -1:
            labels.append(rank)
            continue
        # 順位が付かないデータに関しては最低レベルのラベルを付与
        not_rank = False
        for c in ["中", "取", "除", "降"]:
            if c in str(rank):
                labels.append("low")
                not_rank = True
                break
        if not_rank:
            continue
            
        relative_rank = int(rank) / horse_number
    
        if relative_rank < high:
            labels.append("high")
        elif relative_rank < mid:
            labels.append("middle")
        else:
            labels.append("low")
            
    return labels

In [ ]:
df["label"] = make_label(df["rank"].values, df["total_horse_number_x"].values)
df["rank_1"] = make_label(df["rank_1"].values, df["total_horse_number_x_1"].values)
df["rank_2"] = make_label(df["rank_2"].values, df["total_horse_number_x_2"].values)
df["rank_3"] = make_label(df["rank_3"].values, df["total_horse_number_x_3"].values)

In [ ]:
input_columns = ["race_course", "weather", "ground_status", 
                 "where_racecourse", "race_class", "running_condition", 
                 "frame_number", "horse_number",
                 "sex", "age", "burden_weight", "rider_id", 
                 "tamer_id", "horse_weight", "odds", "popular",
                 "rank_1", "rank_2", "rank_3", 
                 "goal_time_1", "goal_time_2", "goal_time_3",
                 "last_time_1", "last_time_2", "last_time_3", 
                 "kyakusitu_1", "kyakusitu_2", "kyakusitu_3", 
                 "prize_1", "prize_2", "prize_3", "label"]

In [ ]:
# one-hot
df = pd.get_dummies(df[input_columns])

## 学習

In [ ]:
# 学習に用いるデータセットの作成
x = np.array(df)
y = np.array(df[["label_high", "label_middle", "label_low"]])
#del df
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=11)
del x, y
print(x_train.shape)
print(x_test.shape)

In [ ]:
df.head()

In [ ]:
# データセットのシャッフルとバッチ化
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(1024)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1024)

In [ ]:
import utils.sample_model as model
    
# モデルのインスタンスを作成
model = model.HorseModel(x_train.shape[1], 3)

In [ ]:
metrics = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.AUC(name='auc'),
]
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=metrics)

In [ ]:
model.fit(train_ds, epochs=1) 

In [ ]:
model.evaluate(test_ds, verbose=2)

In [ ]:
# テストデータの予測値と正解ラベルの確認
#for x, y in zip(x_test, y_test):
#    print(f"pred: {model.predict(x.reshape(1, -1))}, label: {y}")

In [ ]:
# モデルの保存
# io_m.save_model(model, model_name="first_model")

In [ ]:
# 保存したモデルに不具合がないか確認
# model = io_m.read_model("first_model")
# model.evaluate(test_ds, verbose=2)